# Chapter 11: Encapsulation & Best Practices

Encapsulation is one of the fundamental principles of object‑oriented programming. It refers to the bundling of data and methods that operate on that data within a single unit (a class), and controlling access to that data from the outside. Proper encapsulation hides the internal state of an object and only exposes what is necessary through a well‑defined interface. This leads to more maintainable, robust, and flexible code.

In this chapter, we'll revisit encapsulation in depth, moving beyond the basics. You'll learn:

- The principles of **information hiding** and why it matters.
- How to use **access modifiers** effectively to control visibility.
- The role of **properties** in encapsulation (validation, computed values, read‑only/init‑only).
- The difference between `readonly` and `const` fields.
- When to use **static** members vs. instance members.
- Key design principles: **cohesion**, **coupling**, and the **Single Responsibility Principle**.
- Practical guidelines for designing well‑encapsulated classes.

By the end, you'll have a solid understanding of how to create classes that are both safe and easy to use.

---

## 11.1 Understanding Encapsulation and Information Hiding

Encapsulation is often described as "hiding the internal details and exposing only what is necessary." The goal is to prevent external code from relying on implementation details that may change. This is known as **information hiding**.

Consider a simple `BankAccount` class:

```csharp
public class BankAccount
{
    public decimal Balance; // public field – bad encapsulation!
}
```

If `Balance` is a public field, any code can directly change it, possibly setting it to a negative value or bypassing business rules. There's no control.

Encapsulation suggests we make the field `private` and provide controlled access via methods or properties.

```csharp
public class BankAccount
{
    private decimal _balance;

    public decimal GetBalance() => _balance;

    public void Deposit(decimal amount)
    {
        if (amount <= 0) throw new ArgumentException("Amount must be positive");
        _balance += amount;
    }

    public void Withdraw(decimal amount)
    {
        if (amount <= 0) throw new ArgumentException("Amount must be positive");
        if (amount > _balance) throw new InvalidOperationException("Insufficient funds");
        _balance -= amount;
    }
}
```

Now the balance is protected; it can only be modified through methods that enforce business rules. This is encapsulation in action.

### Benefits of Encapsulation

- **Maintainability** – You can change the internal implementation without affecting external code. For example, you might later add logging or validation.
- **Flexibility** – You can add new features (like transaction history) without breaking existing callers.
- **Security** – You prevent invalid states (e.g., negative balance) by controlling all changes.

---

## 11.2 Access Modifiers – A Deeper Look

Access modifiers are the primary tools for implementing encapsulation. Here's a quick recap and guidelines for their use:

| Modifier | Description | Typical Use |
|----------|-------------|-------------|
| `private` | Accessible only within the same class. | Fields, helper methods. Hide implementation details. |
| `protected` | Accessible within the same class and derived classes. | Members that derived classes may need, but not public. |
| `internal` | Accessible anywhere in the same assembly (project). | For sharing within a project but not with external assemblies. |
| `protected internal` | Accessible within the same assembly or derived classes (even in another assembly). | Rare; combination of protected and internal. |
| `private protected` | Accessible within the same class or derived classes, but only if in the same assembly. | Even rarer; limits protected to assembly. |
| `public` | Accessible from any code. | Only for members that are part of the class's public interface. |

### Guidelines

- **Default to `private`.** Make everything private unless you have a reason to expose it.
- **Prefer `private` fields with public properties.** Properties allow you to add logic later without breaking callers.
- **Use `protected` for members that derived classes need, but that are not part of the public API.**
- **Use `internal` for types or members that are only used within your project.** This is especially useful for library code.
- **Keep the public interface minimal.** Every public member is a commitment; think carefully before adding it.

---

## 11.3 Properties – The Gatekeepers

Properties are the recommended way to expose data from a class. They look like fields from the outside but provide the encapsulation of methods.

### Auto‑Implemented Properties

For simple cases where no logic is needed, auto‑implemented properties are concise:

```csharp
public string Name { get; set; }
```

The compiler generates a private backing field. If you later need validation, you can switch to a full property without breaking callers.

### Full Properties with Validation

```csharp
private string _name;
public string Name
{
    get => _name;
    set
    {
        if (string.IsNullOrWhiteSpace(value))
            throw new ArgumentException("Name cannot be empty");
        _name = value;
    }
}
```

### Read‑Only and Init‑Only Properties

- **Read‑only** (no `set` or `private set`) – the value can only be set in the constructor.
- **Init‑only** (`init` accessor) – the value can be set in the constructor or object initializer, but not afterwards. Perfect for immutable objects.

```csharp
public class Person
{
    public string Name { get; init; }
    public int Age { get; init; }
}

var p = new Person { Name = "Alice", Age = 30 }; // OK
// p.Age = 31; // Error – init‑only
```

### Computed Properties

Properties that return a computed value, with no backing field:

```csharp
public class Rectangle
{
    public double Width { get; set; }
    public double Height { get; set; }
    public double Area => Width * Height; // computed
}
```

### Expression‑Bodied Properties

For simple getters, use the `=>` syntax:

```csharp
public string FullName => $"{FirstName} {LastName}";
```

### Property Accessibility

You can apply different access modifiers to get and set:

```csharp
public int Id { get; private set; } // anyone can read, only class can set
```

This is useful for properties that should be publicly readable but only changed internally.

---

## 11.4 Readonly and Const Fields

### `readonly` Fields

A `readonly` field can only be assigned at declaration or in a constructor. It cannot be changed afterwards. This is useful for values that are set once and then remain constant for the lifetime of the object.

```csharp
public class Configuration
{
    public readonly string ConnectionString;

    public Configuration(string connectionString)
    {
        ConnectionString = connectionString; // OK
    }
}

// Later, you cannot change it:
// config.ConnectionString = "new"; // Error
```

`readonly` fields can be instance or static. They provide a stronger guarantee than a property with a private setter because they truly cannot be changed after construction.

### `const` Fields

A `const` field is a compile‑time constant. It is implicitly static, and its value must be known at compile time. `const` can only be used with primitive types and strings.

```csharp
public class MathConstants
{
    public const double Pi = 3.1415926535;
    public const int DaysInWeek = 7;
}
```

`const` values are inlined by the compiler – if you change a `const` in a library, you must recompile all clients.

### `static readonly` vs. `const`

- Use `const` for values that are truly constant and will never change (e.g., mathematical constants).
- Use `static readonly` for values that are determined at runtime but should be shared across all instances (e.g., configuration loaded from a file).

```csharp
public static readonly DateTime StartupTime = DateTime.Now;
```

---

## 11.5 Static vs. Instance Members – When to Use Which

Static members belong to the type itself, not to any instance. They are useful for:

- Utility methods that don't depend on instance state (e.g., `Math.Max`).
- Factory methods (e.g., `Person.Create`).
- Shared data (e.g., a counter for all instances).

Instance members represent the state and behavior of a specific object.

### Guidelines

- **Prefer instance methods** unless the method does not use any instance data.
- **Avoid excessive static members** because they can make code harder to test (static state is global) and reduce flexibility.
- **Use static classes** for groups of related utility methods (e.g., `FileHelper`). Mark the class `static` to prevent instantiation.
- **Be careful with static mutable state** – it can lead to thread‑safety issues and unexpected coupling.

---

## 11.6 Cohesion and Coupling

Two important design concepts related to encapsulation are **cohesion** and **coupling**.

### Cohesion

Cohesion measures how closely the members of a class are related. A highly cohesive class has a single, well‑defined purpose. All its fields and methods work together to achieve that purpose.

- **High cohesion** is good. It makes classes easier to understand, maintain, and reuse.
- **Low cohesion** means a class does many unrelated things, making it hard to understand and change.

Example of low cohesion:

```csharp
public class Utils
{
    public void PrintDocument() { ... }
    public void CalculateSalary() { ... }
    public void SendEmail() { ... }
}
```

These methods have little in common. Better to split into `DocumentPrinter`, `SalaryCalculator`, `EmailSender`.

### Coupling

Coupling measures how dependent one class is on another. **Low coupling** means classes are relatively independent; changes in one class have minimal impact on others. **High coupling** means classes are tightly intertwined, making changes risky and difficult.

- Favor **low coupling**. Use interfaces, dependency injection, and well‑defined public APIs to reduce dependencies.
- Encapsulation helps reduce coupling by hiding implementation details. If you only depend on a class's public interface, you're less affected by internal changes.

---

## 11.7 The Single Responsibility Principle (SRP)

The **Single Responsibility Principle** states that a class should have only one reason to change. In other words, it should have only one job or responsibility. This is closely related to high cohesion.

For example, consider a class that both represents a `Person` and also saves that person to a database. It has two responsibilities: domain logic and persistence. Changes to the database schema would force changes to the `Person` class, even if the domain logic hasn't changed. Better to separate:

```csharp
public class Person
{
    public string Name { get; set; }
    public int Age { get; set; }
}

public class PersonRepository
{
    public void Save(Person person) { ... }
}
```

Now each class has a clear responsibility, and they are loosely coupled.

---

## 11.8 Putting It All Together: A Well‑Encapsulated Class

Let's build a `BankAccount` class that demonstrates all the principles we've discussed:

- Private fields with public properties (some read‑only, some with validation).
- A method that enforces business rules.
- A static field to generate account numbers.
- Proper use of `readonly` and `const`.
- Adherence to SRP – the class focuses on account operations, not on persistence or UI.

```csharp
using System;
using System.Collections.Generic;

namespace Banking
{
    public class BankAccount
    {
        // Constants
        public const string BankName = "Global Bank";

        // Static readonly – generated at runtime
        private static readonly Random _random = new Random();

        // Static field for account number generation (private)
        private static int _lastAccountNumber = 1000;

        // Instance fields – private
        private readonly string _accountNumber;
        private decimal _balance;
        private readonly List<string> _transactionHistory;

        // Public read‑only properties
        public string AccountNumber => _accountNumber;
        public string OwnerName { get; }
        public decimal Balance => _balance;
        public IReadOnlyList<string> TransactionHistory => _transactionHistory.AsReadOnly();

        // Constructor – sets required data, validates
        public BankAccount(string ownerName, decimal initialDeposit)
        {
            if (string.IsNullOrWhiteSpace(ownerName))
                throw new ArgumentException("Owner name is required", nameof(ownerName));
            if (initialDeposit < 0)
                throw new ArgumentException("Initial deposit cannot be negative", nameof(initialDeposit));

            OwnerName = ownerName;
            _balance = initialDeposit;
            _accountNumber = GenerateAccountNumber();
            _transactionHistory = new List<string>();
            AddTransaction($"Account created with initial deposit {initialDeposit:C}");
        }

        // Private static method for generating account numbers
        private static string GenerateAccountNumber()
        {
            _lastAccountNumber++;
            return $"ACC{_lastAccountNumber:D6}";
        }

        // Public methods for operations
        public void Deposit(decimal amount)
        {
            if (amount <= 0)
                throw new ArgumentException("Deposit amount must be positive", nameof(amount));
            _balance += amount;
            AddTransaction($"Deposited {amount:C}");
        }

        public void Withdraw(decimal amount)
        {
            if (amount <= 0)
                throw new ArgumentException("Withdrawal amount must be positive", nameof(amount));
            if (amount > _balance)
                throw new InvalidOperationException("Insufficient funds");
            _balance -= amount;
            AddTransaction($"Withdrew {amount:C}");
        }

        // Private helper
        private void AddTransaction(string description)
        {
            string entry = $"{DateTime.Now:yyyy-MM-dd HH:mm:ss} - {description}";
            _transactionHistory.Add(entry);
        }

        // Override ToString for useful representation
        public override string ToString()
        {
            return $"Account {AccountNumber} – {OwnerName}, Balance: {Balance:C}";
        }
    }
}
```

**Analysis of encapsulation:**

- Fields are `private` or `readonly`. The list of transactions is exposed as `IReadOnlyList` to prevent external modification.
- Properties are read‑only where appropriate (`AccountNumber`, `OwnerName`, `Balance`). `Balance` is computed from the private field.
- The constructor validates input, ensuring the object starts in a valid state.
- Static fields are used for shared data (`_lastAccountNumber`) and a random generator, both `private`.
- The class has a single responsibility: managing a bank account. It doesn't handle persistence or UI.
- Internal implementation can change (e.g., adding logging, changing transaction storage) without affecting external code.

---

## 11.9 Common Pitfalls and Best Practices – Recap

### 1. Avoid Public Fields
Always use properties or methods. Public fields break encapsulation and offer no control.

### 2. Keep Fields Private
Even within a class, consider whether a field needs to be `protected`. Prefer `private` unless derived classes truly need direct access.

### 3. Use Properties for Exposed Data
Properties allow you to add logic later. They also support data binding in UI frameworks.

### 4. Validate Input in Constructors and Property Setters
Ensure objects are never in an invalid state. Throw appropriate exceptions early.

### 5. Prefer Immutability Where Possible
Use `readonly` fields, `init`‑only properties, and records to create immutable objects. This avoids many bugs and simplifies concurrency.

### 6. Don't Expose Collections Directly
If you expose a `List<T>` as a property, callers can modify it. Expose `IEnumerable<T>` or `IReadOnlyList<T>` and provide methods to add/remove if needed.

### 7. Be Mindful of Static State
Static mutable fields introduce global state, which can cause threading issues and hidden dependencies. Use them sparingly and consider thread safety.

### 8. Follow the Single Responsibility Principle
If a class is growing too large, split it. This improves cohesion and reduces coupling.

### 9. Use Access Modifiers to Express Intent
Make members as private as possible. Only increase visibility when necessary.

### 10. Document the Public Interface
Provide XML comments for public members, explaining what they do, parameters, return values, and exceptions. This helps other developers use your class correctly.

---

## 11.10 Chapter Summary

In this chapter, we deepened our understanding of encapsulation – the principle of hiding internal details and exposing a controlled interface. You learned:

- **Information hiding** protects internal state and allows implementation changes without breaking callers.
- **Access modifiers** (`private`, `protected`, `internal`, `public`) are the primary tools for controlling visibility.
- **Properties** provide a field‑like syntax with the power of methods, enabling validation, computed values, and read‑only/init‑only semantics.
- **Readonly** and **const** fields offer different levels of immutability.
- **Static members** belong to the type; use them wisely.
- **Cohesion** and **coupling** are key quality measures – aim for high cohesion and low coupling.
- The **Single Responsibility Principle** guides you to keep classes focused.

With these principles and techniques, you can design classes that are robust, maintainable, and a pleasure to use.

In the next chapter, **Delegates, Events, and Lambda Expressions**, we'll explore how C# enables functional‑style programming and event‑driven architectures. You'll learn how to pass methods as parameters, publish and subscribe to events, and write concise inline functions with lambdas.

**Exercises:**

1. Refactor a class from a previous chapter (e.g., `Person` or `Product`) to improve encapsulation. Make all fields private, add validation in properties, and expose collections as read‑only.
2. Create a `Temperature` class that stores temperature in Celsius internally but provides properties for Celsius, Fahrenheit, and Kelvin. Ensure that setting any property updates the internal value, and that invalid temperatures (below absolute zero) are rejected.
3. Design a `ShoppingCart` class that contains a private list of items. Provide methods to add, remove, and calculate total. Ensure that the list cannot be modified directly from outside.
4. Add a static counter to a class that tracks how many instances have been created. Make it `private` and expose a static read‑only property.
5. Identify a class in your own code (or a previous exercise) that violates the Single Responsibility Principle. Refactor it into two or more classes.

Now, get ready to dive into delegates and events in Chapter 12!